In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import torch.nn as nn
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [2]:
cd C:/Users/82102/Desktop/Datasets(Right) 최종-20220824T025446Z-001/Datasets(Right) 최종

C:\Users\82102\Desktop\Datasets(Right) 최종-20220824T025446Z-001\Datasets(Right) 최종


In [3]:
def distance(x1, y1, x2, y2):
    m = (y2-y1) / (x2-x1)
    return m

In [4]:
def Coefficient(df): 
    coef = []
    for i in range(49, 351):
        frame1 = df[i][gradlab1][0:2]
        frame2 = df[i][gradlab2][0:2]
        coef.append(distance(frame1[0], frame1[1], frame2[0], frame2[1]))
    coef = pd.DataFrame(coef, columns= ['coef'])
    
    q3=coef['coef'].quantile(0.75)
    q1=coef['coef'].quantile(0.25)
    iqr=q3-q1
    outlier_top=q3+1.5*iqr
    outlier_bottom=q1-1.5*iqr
    idx=coef[(coef['coef']<outlier_bottom) | (coef['coef']>outlier_top)].index
    
    idx_list=[]
    for i in idx:
        idx_list.append(str(i))
    idx_list.append('end')

    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    outlier_str = idx_string_split[0].strip()
    outlier = list(map(int, outlier_str.split(' ')[:-1]))

    for i in range(len(coef)-1):
        if i in outlier:
            if i == 0:
                continue
            elif i != (len(coef)-1):
                coef['coef'][i] = (coef['coef'][i-1] + coef['coef'][i+1])/2
            else:
                break
                
    coef = np.array(coef)[1:301]
    
    return coef


In [5]:
def Acceleration(pos_1, array_1, acclab):
    df = pd.DataFrame(array_1[:,acclab,:], columns=['x','y','z'])
    idx_list=[]
    for i in ['x','y','z']:
        q3=df[i].quantile(0.75)
        q1=df[i].quantile(0.25)
        iqr=q3-q1
        outlier_top=q3+1.5*iqr
        outlier_bottom=q1-1.5*iqr

        idx=df[(df[i]<outlier_bottom) | (df[i]>outlier_top)].index
        for i in idx:
            idx_list.append(str(i))
        idx_list.append('end')
    
    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    x_outlier_str = idx_string_split[0].strip()
    y_outlier_str = idx_string_split[1].strip()
    x_outlier = list(map(int, x_outlier_str.split(' ')[:-1]))
    y_outlier = list(map(int, y_outlier_str.split(' ')[:-1]))
    
    acc_1_parsed = array_1[:,acclab,:]
    for i in range(len(pos_1)-1):
        if i in x_outlier:
            acc_1_parsed[i][0] = (acc_1_parsed[i-1][0] + acc_1_parsed[i+1][0])/2 
        if i in y_outlier:
            acc_1_parsed[i][1] = (acc_1_parsed[i-1][1] + acc_1_parsed[i+1][1])/2 
       
    final_df = pd.DataFrame(acc_1_parsed, columns = ['x','y','z'])
    acc_array = StandardScaler().fit_transform(final_df)
    acc_array = acc_array[49:349,:]
    
    return acc_array[:,:2] 

In [6]:
def preprocessing(gesture,num_session):
    data2 = np.zeros((1,300,6))
    for i in range(1, num_session+1):
        data = np.zeros((300,1))
        pos = np.load('%s_%s_p_%d.npy' % (hand, gesture, i))
        acc = np.load('%s_%s_a_%d.npy' % (hand, gesture, i))
        acc1 = Acceleration(pos, acc, acclab1)
        acc2 = Acceleration(pos, acc, acclab2)
        coef = Coefficient(pos)
        if gesture == right_gesture:
            label = np.ones((300,1))
        else:
            label = np.zeros((300,1))
        data = np.hstack((data,acc1,acc2,coef,label))
        data = data[:, 1:].reshape((1,300,6))
        data2 = np.vstack((data2,data))
        
    return data2

In [7]:
num_session = 21
gesture_list = ['URF','UMF','ULF','DLF','DMF','DRF','URB','UMB','ULB','DLB','DMB','DRB','URO','ULO','DLO','DRO']
right_gesture = gesture_list[0]
hand = 'Right'

acclab1 = 8
acclab2 = 12
gradlab1 = 8
gradlab2 = 12

In [ ]:
data = np.vstack((target_data[1:], error_data[1:]))
data.shape

In [ ]:
np.save('data_URF_DLB.npy', data)

In [ ]:
gesture_list 

In [21]:
error_data = np.empty((1,300,6))
gesture_list = ['URF','UMF','ULF','DLF','DMF','DRF','URB','UMB','ULB','DLB','DMB','DRB','URO','ULO','DLO','DRO']
target_data = preprocessing(gesture_list[0],25)
gesture_list.pop(0)
for j in range(15):
    error_data = np.vstack((error_data,preprocessing(gesture_list[j],1)[1:]))

In [22]:
target_data.shape, error_data.shape

((26, 300, 6), (16, 300, 6))

In [25]:
error_data[1:]

array([[[-0.2803368 ,  0.02026211,  0.87871036,  1.26208221,
          6.01742496,  0.        ],
        [-0.74370233, -0.70692696,  0.92014241,  0.29219323,
          7.15660883,  0.        ],
        [ 0.66182169,  0.77048402,  0.24068244,  0.51286197,
          4.33856799,  0.        ],
        ...,
        [ 1.13963415,  0.79638294,  0.34046096,  0.14146042,
          3.56285254,  0.        ],
        [-0.6116564 , -0.74727352, -0.94227983, -0.38867595,
          4.00234801,  0.        ],
        [-1.62595564, -1.00478409, -0.62659917, -0.85916246,
          5.00715596,  0.        ]],

       [[ 2.34758992,  2.254597  ,  0.69870308,  0.09413947,
          3.59444015,  0.        ],
        [-1.40562749, -1.57125402, -0.85284586,  0.2117603 ,
          3.62727783,  0.        ],
        [ 0.39701684,  0.3147265 ,  1.69117772, -0.08931622,
          3.60416548,  0.        ],
        ...,
        [-0.26470746,  1.17301308, -1.66577155, -0.86418833,
          2.38172137,  0.        ],
  

In [26]:
target_data[1:]

array([[[-9.00372812e-01, -4.19423456e-01,  9.44906915e-01,
          7.99437642e-01, -1.57831751e+01,  1.00000000e+00],
        [-3.32484235e-01, -8.64761249e-02,  1.05114560e+00,
          1.17093223e+00, -6.84319743e+00,  1.00000000e+00],
        [ 1.27287226e+00,  1.46969375e+00,  1.18887804e-01,
          1.01647564e-02,  2.09678023e+00,  1.00000000e+00],
        ...,
        [-2.09570158e+00,  1.27662747e+00, -2.22521911e+00,
         -1.95951852e+00,  1.57005377e+00,  1.00000000e+00],
        [ 8.49257608e-02,  1.10908138e+00,  2.41333253e+00,
         -1.80648271e+00, -9.36748478e-01,  1.00000000e+00],
        [ 1.97634944e-01,  9.41535286e-01, -1.67184151e+00,
         -1.65344691e+00,  1.72804132e+00,  1.00000000e+00]],

       [[ 2.16896375e-01,  2.10021455e+00, -1.50500668e+00,
         -1.64138991e+00,  2.61205505e+00,  1.00000000e+00],
        [ 1.20883276e-01, -2.45661133e-01, -7.03490858e-02,
          3.82926780e-01,  1.90768866e+00,  1.00000000e+00],
        [-1.42856

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, file_name, transform=None,  target_transform=None):
        df = np.load(file_name)
        self.x_data = df[:,:,:-1]
        self.y_data = df[:,:,-1]
        #self.transform = transform
        #self.target_transform = target_transform
        

    def __getitem__(self, idx):
        x_data = torch.FloatTensor([self.x_data[idx]])
        y_data = torch.Tensor(self.y_data[idx])
        
        return x_data, y_data
        
    def __len__(self):
        return len(self.x_data) #데이터셋 크기 리턴
               
        #i번째 샘플을 찾는데 사용됨. 

In [ ]:
dataset = CustomDataset('data_URF_DLB.npy')
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True)
val_dataloader = DataLoader(val_dataset, shuffle = True)
test_dataloader = DataLoader(test_dataset, shuffle = True)

In [ ]:
train_x, train_y = next(iter(train_dataloader))
train_x = train_x.squeeze(dim=1)
train_y = train_y[:,-1].unsqueeze(dim=1)
train_x.size(), train_y.size()

In [ ]:
batch_size = 25 
input_size= 5
hidden_size = 64
num_layers = 2
dropout_rate= 0.5
num_classes = 1

In [ ]:
class LSTM_Chicka(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, dropout_rate):
        super(LSTM_Chicka,self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.dropout_rate = dropout_rate
        self.lstm =  nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout_rate, batch_first = True)
        self.layer_out = nn.Linear(hidden_size, num_classes) 
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        out, _status = self.lstm(x)
        out = self.layer_out(out[:, -1]) 
        out = self.sigmoid(out)

        return out

In [ ]:
model = LSTM_Chicka(num_classes, input_size, hidden_size, num_layers, dropout_rate)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer= optimizer)

In [ ]:
val_x, val_y = next(iter(val_dataloader))
val_x = val_x.squeeze(dim=1)
val_y = val_y[:,-1].unsqueeze(dim=1)
val_x.size(), val_y.size()

In [ ]:
train_iterations = []
train_losses = []
val_iterations = []
val_losses = []

In [ ]:
def train(num_epochs):

    print("Begin training...") 
    for epoch in range(1, num_epochs+1): 
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_vall_loss = 0.0 
        total = 0 
        
         # Training Loop 
        for data in train_dataloader: 
        #for data in enumerate(train_loader, 0): 
            inputs, outputs = data  # get the input and real species as outputs; data is a list of [inputs, outputs] 
            inputs = inputs.squeeze(dim=1)
            outputs = outputs[:,-1].unsqueeze(dim=1)
            optimizer.zero_grad()   # zero the parameter gradients          
            predicted_outputs = model(inputs)   # predict output from the model 
            train_loss = criterion(predicted_outputs, outputs)   # calculate loss for the predicted output  
            train_loss.backward()   # backpropagate the loss 
            optimizer.step()        # adjust parameters based on the calculated gradients 
            running_train_loss +=train_loss.item()  # track the loss value 
        
        # Calculate training loss value 
        train_loss_value = running_train_loss/len(train_dataloader)
        train_iterations.append(epoch)
        train_losses.append(train_loss_value)
            
        # Validation Loop 
        with torch.no_grad(): 
            model.eval() 
            for data in val_dataloader: 
                inputs, outputs = data
                inputs = inputs.squeeze(dim=1)
                outputs = outputs[:,-1].unsqueeze(dim=1)
                predicted_outputs = model(inputs) 
                val_loss = criterion(predicted_outputs, outputs) 
             
               # The label with the highest value will be our prediction 
                _, predicted = torch.max(predicted_outputs, 1) 
                running_vall_loss += val_loss.item()  
                total += outputs.size(0) 
                running_accuracy += (predicted == outputs).sum().item() 
        
        # Calculate validation loss value 
        val_loss_value = running_vall_loss/len(val_dataloader) 
        val_iterations.append(epoch)
        val_losses.append(val_loss_value)        
        # Calculate accuracy as the number of correct predictions in the validation batch divided by the total number of predictions done.  
        accuracy = (100 * running_accuracy / total)     
 
        # Save the model if the accuracy is the best 
        #if accuracy > best_accuracy:
        #    best_accuracy = accuracy 
         
        # Print the statistics of the epoch 
        print('Completed training batch', epoch, 'Training Loss is: %.4f' %train_loss_value, 'Validation Loss is: %.4f' %val_loss_value, 'Accuracy is %d %%' % (accuracy))

In [ ]:
train(100)

In [ ]:
plt.plot(range(1, len(train_iterations)+1), train_losses, 'b--')
plt.plot(range(1, len(val_iterations)+1), val_losses, 'b--')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.xlim(0, len(train_iterations)+1) # 일정한 scale
plt.grid(True)
plt.tight_layout()
plt.show()